In [16]:
import gcsfs
import pandas as pd
import numpy as np
import re
import openpyxl

In [2]:
# Define path to NCEI metadata and get system path with gcsfs
path = 'noaa-passive-bioacoustic/big_query_metadata'
fs = gcsfs.GCSFileSystem(path)

In [38]:
file_list = fs.ls(path)
allData = pd.DataFrame()
# Iterate over files. Currently won't read .xlsx, only .csv
for file in file_list[1:]:
    print(file)
    with fs.open(file,'rb') as f:
#         if 'xlsx' in file:
#             df = pd.read_excel(f,sheet_name=1)
        if 'csv' in file:
            df = pd.read_csv(f)
            
            # Concatenate all dataframes into single dataframe
            if 'SHAPE' in df.columns:
                allData = pd.concat([allData,df[ ["FILE_NAME" , "START_DATE", "START_TIME", df.filter(like='RATE').columns[0], 
                                                 "SENSOR_DEPTH", "SHAPE"] ] ], axis=0, ignore_index=True )
            else:
                allData = pd.concat([allData,df[ ["FILE_NAME" , "START_DATE", "START_TIME", df.filter(like='RATE').columns[0], 
                                                 "SENSOR_DEPTH", "LAT", "LON" ] ] ], axis=0, ignore_index=True ) 

noaa-passive-bioacoustic/big_query_metadata/NCEI_ADEON_PAD_metadata.xlsx


In [ ]:
# Parser for lat/lon formats
def cruise_track_parser(s: str):
    i = 0
    j = len(s) - 1
    while s[i] != "(":
        i += 1
    while s[j] != ")":
        j -= 1
    s = s[i+1:j]
    s = s.split(",")
    output = []
    sep = re.compile(r"[\s\t]+")
    for entry in s:
        e = re.split(sep, entry.strip())
        output.append([float(e[0][1:]), float(e[1][:-1])])
    return np.array(output)


In [ ]:
# Modify data for consistent formatting

MIN_LAT = np.zeros(allData.shape[0])
MIN_LON = np.zeros(allData.shape[0])
MAX_LAT = np.zeros(allData.shape[0])
MAX_LON = np.zeros(allData.shape[0])
SAMPLE_RATE = np.zeros(allData.shape[0])
SENSOR_DEPTH = np.zeros(allData.shape[0])

for index, row in allData.iterrows():
        
    if type(row['SHAPE'])==float:
        minLat = row['LAT']
        maxLat = row['LAT']
        minLon = row['LON']
        maxLon = row['LON']
        
    elif 'MULTIPOINT' in row['SHAPE']:
        tmp = cruise_track_parser(row['SHAPE'])
        minLat = np.min(tmp[:,1])
        maxLat = np.max(tmp[:,1])
        minLon = np.min(tmp[:,0])
        maxLon = np.max(tmp[:,0])
        
    elif 'POINT' in row['SHAPE']:
        tmp = cruise_track_parser(row['SHAPE'])
        lat = tmp[0][1] # row['SHAPE'].split(' ')[1][2:]
        lon = tmp[0][0] # row['SHAPE'].split(' ')[2][:-2]
        minLat = float(lat)
        maxLat = float(lat)
        minLon = float(lon)
        maxLon = float(lon)
        
    MIN_LAT[index] = minLat
    MAX_LAT[index] = maxLat
    MIN_LON[index] = minLon
    MAX_LON[index] = maxLon
    
    if np.isnan(row['SAMPLE_RATE']):
        SAMPLE_RATE[index] = row['SAMPLE_RATE_Hz']
    else:
        SAMPLE_RATE[index] = 1000*row['SAMPLE_RATE']
        
    if row['SENSOR_DEPTH'] < 0:
        SENSOR_DEPTH[index] = abs(row['SENSOR_DEPTH'])


In [ ]:
# Drop unneeded columns and replace others with new formatting
allData = allData.drop(['SHAPE','LAT','LON', 'SAMPLE_RATE_Hz','SAMPLE_RATE','SAMPLE_RATE_Hz','SENSOR_DEPTH'],axis=1)
allData['MIN_LAT'] = MIN_LAT
allData['MAX_LAT'] = MIN_LAT
allData['MIN_LON'] = MIN_LON
allData['MAX_LON'] = MAX_LON
allData['SAMPLE_RATE'] = SAMPLE_RATE
allData['SENSOR_DEPTH'] = SENSOR_DEPTH